# Epochs and Julian Dates

The representation of an epoch, that is of a specific point in time, be it in the future or in the past, can be rather confusing. In `pykep` we opted to offer the dedicated class {class}`pykep.epoch` that takes care to offer a simple interface and, under the hoods, interfaces seamlessly both to the c++ `std::chrono`  library and the python {class}`numpy.datetime` module. Find it in our API documentation under the section "[Relatated to epochs](../epoch.rst)"

Let us briefly show its interface.

An `epoch` may be created in one of four ways: 

1. constructing one from a Julian Date (i.e. a float representing the number of days passed from some historical date).
2. constructing one from a datetime object.
3. constructing directly from an ISO 8601 string.
4. requesting the current date from the {func}`pykep.utc_now` function.

:::{note}
**MJD2000** is the Default Julian Date. When not specified othewise by the user, in the context of epoch arithmetics a float will always be considered by `pykep` as a Modified Julian Date 2000, i.e. as the number of days from `2000-01-01T00:00:00.000000`, or as days if it represents a duration.

:::{note}
The date in `pykep` **does** account for leap seconds. If the user wishes to use the exact ISO 8601 representation of some epoch, also including leap seconds, he will have to account for the offset himself. As of of 2023 this may account to maximum 28 seconds. [More info](https://en.wikipedia.org/wiki/Leap_second) on leap seconds.

In [ ]:
import pykep as pk
import datetime

## Julian dates

In [ ]:
ep = pk.epoch(0.)

we can print this on screen:

In [ ]:
print(ep)

.. or instantiate an epoch by explicitly mentioning the Julian Date type:

In [ ]:
ep = pk.epoch(0., pk.epoch.julian_type.MJD2000)
print(ep)

.. or use a different Julian Date than the default MJD2000:

In [ ]:
ep = pk.epoch(2460676.5000000, pk.epoch.julian_type.JD)
print(ep)

:::{note}
`pykep` supports the following Julian Dates MJD2000 (the default), MJD and JD. (see {class}`pykep.epoch.julian_type`)

We may also request an epoch corresponding to the current UTC time:

In [ ]:
ep = pk.utc_now()
print(ep)

or construct it from an iso string:


In [ ]:
ep = pk.epoch("2023-10-28T00:01:02.12")
print(ep)


## Datetime interoperability

If we have a datetime object from python builtin datetime library we can construct an epoch with it:

In [ ]:
dt = datetime.datetime(year=2033, month=11, day=12, hour=12, minute=22, second=12, microsecond=14532)
ep = pk.epoch(dt)
print(ep)

and convert it, when needed, to a julian representation:

In [ ]:
print(ep.mjd)

## The epoch math
Additions and subtractions are allowed between epochs and floats or `datetime.timedelta`. 
When using floats days are always assumed.

In [ ]:
ep = pk.epoch(0)
ep = ep + 21.2353525 # This will be interpreted as days
print(ep)

In [ ]:
ep = pk.epoch(0)
ep = ep + datetime.timedelta(hours = 5, seconds=54, days=21, minutes=38, microseconds=456000) # This will be interpreted as days
print(ep)

Some basic comparison operators are also allowed and may turn handy!

In [ ]:
print(ep < ep + 4)
print(ep == ep + datetime.timedelta(days=32) - 32)

In [ ]:
from sgp4.api import Satrec
from sgp4 import exporter
import pykep as pk
import numpy as np

class tle:
    """__init__(line1, line2)

    This User Defined Planet (UDPLA) represents a satellite orbiting the Earth and defined in the TLE format
    and propagated using the SGP4 propagator.

    .. note::
       The resulting ephemerides will be returned in SI units and in the True Equator Mean Equinox (TEME) reference frame

    """
    def __init__(self, line1, line2):
        import pykep as pk
        self.satellite = Satrec.twoline2rv(line1, line2)
        self.e = 0
        self.ref_epoch = pk.epoch(self.satellite.jdsatepoch + self.satellite.jdsatepochF, pk.epoch.julian_type.JD)
    def eph(self, ep):
        jd = ep.jd
        jd_i = int(jd)
        jd_fr = jd-jd_i
        self.e, r, v = self.satellite.sgp4(jd_i, jd_fr)
        return [[it*1000 for it in r], [it*1000 for it in v]]
    def eph_v(self, eps):
        jds = [ep.jd for ep in eps]
        jd_is = [int(item) for item in jds]
        jd_frs = [a-b for a,b in zip(jds,jd_is)]
        self.e, r, v = self.satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))
        rv = np.hstack((r,v))
        return rv.flatten() 
    def get_name(self):
        return self.satellite.satnum_str + " - SGP4"
    def get_extra_info(self):
        line1, line2 = exporter.export_tle(self.satellite)
        return "TLE line1: " + line1 + "\nTLE line2: " + line2 
    def get_mu_central_body(self):
        return pk.MU_EARTH

In [ ]:
line1 = '1 25544U 98067A   19343.69339541  .00001764  00000-0  38792-4 0  9991'
line2 = '2 25544  51.6439 211.2001 0007417  17.6667  85.6398 15.50103472202482'
udpla = tle(line1, line2)
eps = [pk.epoch(when%1000.2) for when in range(100000)]
satellite = Satrec.twoline2rv(line1, line2)


In [ ]:
pla = pk.planet(udpla)

In [ ]:
%%time
for ep in eps:
    udpla.eph(ep)

In [ ]:
%%time
for ep in eps:
    pla.eph(ep)

In [ ]:
satellite = Satrec.twoline2rv(line1, line2)

In [ ]:
%%timeit
pla.eph_v(eps)

In [ ]:
%%timeit
for ep in eps:
    jd = ep.jd
    jd_i = int(jd)
    jd_fr = jd-jd_i
    e, r, v = satellite.sgp4(jd_i, jd_fr)
    [it*1000 for it in r], [it*1000 for it in v]

In [ ]:
%%timeit
satellite.sgp4(27000, 0.0)

In [ ]:
jds = [item.jd for item in eps]
jd_is = [int(item) for item in jds]
jd_frs = [a-b for a,b in zip(jds, jd_is)]
e, r, v = satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))


In [ ]:
r

In [ ]:
type(r[0])

In [ ]:
np.hstack((r,v))

In [ ]:
r[0]

In [ ]:
v[0]